## Sentiment Analysis Project for Twitter
_____

# Twitter Spark Streaming
Set up the credentials for a twitter app at https://apps.twitter.com/
    

In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
# May cause deprecation warnings, safe to ignore, they aren't errors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [4]:
# Can only run this once. restart your kernel for any errors.
sc = SparkContext()

In [5]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

In [6]:
#creating a sockets for the communication with spark
socket_stream = ssc.socketTextStream("127.0.0.1", 9090)

In [7]:
lines = socket_stream.window( 20 ) # the window seconds we have to wait

In [8]:
# cleaning the data coming a with a line and searching for a specific word
( lines.flatMap( lambda text: text.split( "\n" )).filter( lambda word: '*' in word ).foreachRDD( lambda rdd: rdd.toDF() ) ) #Lower cases the word
#words.saveAsTextFiles('tempFile.txt')

### This part is what is going to make a connection with the server 


In [9]:
# vaderSentiment is sentiment analysis function to analyze the tweets
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [10]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json
import string
import re
%matplotlib inline 

In [11]:
#start the streaming
ssc.start()

In [12]:
#stop the streaming
ssc.stop()

In [13]:
# writting a file with all the twiting all the data form twitter
tweets_data = []
# name of file
OUTPUT_FILE = "tempFile.txt"
# readind file. and saving it to tweets list
with open(OUTPUT_FILE, "r") as tweets_file:
    for line in tweets_file:
        tweet = json.loads(line)
        tweets_data.append(tweet)

In [14]:
#creating dataframe
dfTwitter = pd.DataFrame(tweets_data, columns=['created_at','langu', 'text', 'source'])
dfTwitter.head()

,created_at,langu,text,source
0,Wed Nov 04 23:58:20 +0000 2020,NaN,@chuuxp * pulls out push pop weopon *,"<a href=""http://twitter.com/download/android"" ..."
1,Wed Nov 04 23:58:24 +0000 2020,NaN,RT @donelson52: * weeps * \n\nIndiana college ...,"<a href=""http://twitter.com/download/iphone"" r..."
2,Wed Nov 04 23:58:27 +0000 2020,NaN,RT @xsdvoid: ┊┊┊┊⋆ ✧ · ✧ ▪\n┊┊┊☆ * * ⋆\...,"<a href=""http://twitter.com/download/android"" ..."
3,Wed Nov 04 23:58:27 +0000 2020,NaN,"RT @tinazhxng: if you are from pennsylvania, w...","<a href=""http://twitter.com/download/iphone"" r..."
4,Wed Nov 04 23:58:28 +0000 2020,NaN,RT @k2spritebeans: nevada be like,"<a href=""http://twitter.com/download/iphone"" r..."


In [15]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    #text = re.sub('[0-9]+', '', text)
    return text

In [16]:
dfTwitter['text'] = dfTwitter['text'].apply(lambda x: remove_punct(x))
dfTwitter.head(10)

,created_at,langu,text,source
0,Wed Nov 04 23:58:20 +0000 2020,NaN,chuuxp pulls out push pop weopon,"<a href=""http://twitter.com/download/android"" ..."
1,Wed Nov 04 23:58:24 +0000 2020,NaN,RT donelson52 weeps \n\nIndiana college stud...,"<a href=""http://twitter.com/download/iphone"" r..."
2,Wed Nov 04 23:58:27 +0000 2020,NaN,RT xsdvoid ┊┊┊┊⋆ ✧ · ✧ ▪\n┊┊┊☆ ⋆\n┊┊★...,"<a href=""http://twitter.com/download/android"" ..."
3,Wed Nov 04 23:58:27 +0000 2020,NaN,RT tinazhxng if you are from pennsylvania wisc...,"<a href=""http://twitter.com/download/iphone"" r..."
4,Wed Nov 04 23:58:28 +0000 2020,NaN,RT k2spritebeans nevada be like,"<a href=""http://twitter.com/download/iphone"" r..."
5,Wed Nov 04 23:58:29 +0000 2020,NaN,RT WizardsCash Sending a 25 gift to someone wh...,"<a href=""http://twitter.com/download/android"" ..."
6,Wed Nov 04 23:58:29 +0000 2020,NaN,restricted,"<a href=""http://twitter.com/download/iphone"" r..."
7,Wed Nov 04 23:58:35 +0000 2020,NaN,QuantessentialR or however many representativ...,"<a href=""http://twitter.com/download/android"" ..."
8,Wed Nov 04 23:58:38 +0000 2020,NaN,Something good \n\nThe brain lag,"<a href=""http://twitter.com/download/android"" ..."
9,Wed Nov 04 23:58:40 +0000 2020,NaN,RT k2spritebeans nevada be like,"<a href=""http://twitter.com/download/iphone"" r..."


In [17]:
#if we want to check all the tweets
for i in dfTwitter['text']:
    print(i)

chuuxp  pulls out push pop weopon 
RT donelson52  weeps  

Indiana college student with COVID19 dies in her dorm room  CBS News httpstco5UafGK87IR
RT xsdvoid ┊┊┊┊⋆ ✧　 　 · 　 ✧ ▪
┊┊┊☆ 　　  ⋆
┊┊★   welcome to my little world
┊┊  ☆  
┊☆ ° ✧　 　 ·
★ httpstco7JvFi37n40
RT tinazhxng if you are from pennsylvania wisconsin michigan and georgia PLEASE CHECK THE STATUS OF UR BALLOT AND MAKE SURE IT WAS COU…
RT k2spritebeans nevada be like
RT WizardsCash Sending a 25 gift to someone who 

 RTFollows Me
 Comment a new crypto token amp ticker hashtag

Ends in 24 hours
Dap…
restricted 
QuantessentialR  or however many representatives they have httpstcobygvg9G8q6
Something good  

The brain lag
RT k2spritebeans nevada be like
RT GaganGulati333 Halloween Theme  Giveaway 👻

Rules 

1 Follow mreeves1610 BPost222  and GaganGulati333

2 Like and Retweet this t…
ok now my header is just chefs kiss 
RT Hotbitnews HotbitAMA
 
Glad to invite Przeme Karda CEO amp Cofounder of TecraCoin to join AMA at 700 UTC on N

In [18]:
analyzer = SentimentIntensityAnalyzer()

In [19]:
dfTwitter.shape

(501, 4)

In [20]:
# This function is where I perform the sentiment analysis on each tweets form the Spark 
num = 0
myList = []
for sentence in dfTwitter['text']:
    print( num, " " , analyzer.polarity_scores(sentence))
    myList.append(analyzer.polarity_scores(sentence))
    num +=1

0   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
1   {'neg': 0.138, 'neu': 0.862, 'pos': 0.0, 'compound': -0.34}
2   {'neg': 0.0, 'neu': 0.889, 'pos': 0.111, 'compound': 0.4588}
3   {'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'compound': 0.7241}
4   {'neg': 0.0, 'neu': 0.615, 'pos': 0.385, 'compound': 0.3612}
5   {'neg': 0.0, 'neu': 0.888, 'pos': 0.112, 'compound': 0.4404}
6   {'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound': -0.3818}
7   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
8   {'neg': 0.289, 'neu': 0.361, 'pos': 0.349, 'compound': 0.128}
9   {'neg': 0.0, 'neu': 0.615, 'pos': 0.385, 'compound': 0.3612}
10   {'neg': 0.106, 'neu': 0.78, 'pos': 0.115, 'compound': 0.0516}
11   {'neg': 0.0, 'neu': 0.545, 'pos': 0.455, 'compound': 0.6124}
12   {'neg': 0.0, 'neu': 0.702, 'pos': 0.298, 'compound': 0.7579}
13   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
14   {'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'compound': 0.7241}
15   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, '

358   {'neg': 0.125, 'neu': 0.461, 'pos': 0.414, 'compound': 0.6597}
359   {'neg': 0.268, 'neu': 0.732, 'pos': 0.0, 'compound': -0.6597}
360   {'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}
361   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
362   {'neg': 0.0, 'neu': 0.772, 'pos': 0.228, 'compound': 0.5499}
363   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
364   {'neg': 0.343, 'neu': 0.657, 'pos': 0.0, 'compound': -0.886}
365   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
366   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
367   {'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'compound': 0.4648}
368   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
369   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
370   {'neg': 0.0, 'neu': 0.934, 'pos': 0.066, 'compound': 0.0516}
371   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
372   {'neg': 0.0, 'neu': 0.263, 'pos': 0.737, 'compound': 0.4215}
373   {'neg': 0.0, 'neu': 0.881, 'pos': 0

In [21]:
len(myList)

501

In [22]:
dfTwitter["Result"] =  myList

In [23]:
dfTwitter.dtypes

created_at     object
langu         float64
text           object
source         object
Result         object
dtype: object

In [24]:
mydict = dict(dfTwitter["Result"])

In [25]:
dfTwitter["Analysis Result"] = 0

In [26]:
for x in mydict:
    if (mydict[x]['neg'] > mydict[x]['neu'] and mydict[x]['neg'] >mydict[x]['pos'] and mydict[x]['neg'] >mydict[x]['compound']):
        print (dfTwitter["text"].iloc[x])
        print("----Tweet is negative, according to the score", mydict[x]['neg'])
        print("***", mydict[x], "***\n")
        dfTwitter["Analysis Result"].iloc[x]= "negative"
    elif (mydict[x]['neu'] > mydict[x]['neg'] and mydict[x]['neu'] >mydict[x]['pos'] and mydict[x]['neu'] >mydict[x]['compound']):
        print (dfTwitter["text"].iloc[x])
        print("----Tweet is neutral, according to the score", mydict[x]['neu'])
        print("***", mydict[x], "***\n")
        dfTwitter["Analysis Result"].iloc[x]= "neutral"
    elif (mydict[x]['pos'] > mydict[x]['neg'] and mydict[x]['pos'] >mydict[x]['neu'] and mydict[x]['pos'] >mydict[x]['compound']):
        print (dfTwitter["text"].iloc[x])
        print("----Tweet is positive, according to the score", mydict[x]['pos'])
        print("***", mydict[x], "***\n")
        dfTwitter["Analysis Result"].iloc[x]= "positive"
    elif (mydict[x]['compound'] > mydict[x]['neg'] and mydict[x]['compound'] >mydict[x]['neu'] and mydict[x]['compound'] >mydict[x]['pos']):
        print (dfTwitter["text"].iloc[x])
        print("----Tweet is Compound, according to the score", mydict[x]['compound'])
        print("***", mydict[x], "***\n")
        dfTwitter["Analysis Result"].iloc[x]= "Compound"
    else :
        print("****something went wrong****")

chuuxp  pulls out push pop weopon 
----Tweet is neutral, according to the score 1.0
*** {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} ***

RT donelson52  weeps  

Indiana college student with COVID19 dies in her dorm room  CBS News httpstco5UafGK87IR
----Tweet is neutral, according to the score 0.862
*** {'neg': 0.138, 'neu': 0.862, 'pos': 0.0, 'compound': -0.34} ***

RT xsdvoid ┊┊┊┊⋆ ✧　 　 · 　 ✧ ▪
┊┊┊☆ 　　  ⋆
┊┊★   welcome to my little world
┊┊  ☆  
┊☆ ° ✧　 　 ·
★ httpstco7JvFi37n40
----Tweet is neutral, according to the score 0.889
*** {'neg': 0.0, 'neu': 0.889, 'pos': 0.111, 'compound': 0.4588} ***

RT tinazhxng if you are from pennsylvania wisconsin michigan and georgia PLEASE CHECK THE STATUS OF UR BALLOT AND MAKE SURE IT WAS COU…
----Tweet is neutral, according to the score 0.784
*** {'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'compound': 0.7241} ***

RT k2spritebeans nevada be like
----Tweet is neutral, according to the score 0.615
*** {'neg': 0.0, 'neu': 0.615, 'pos': 0.385, 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)



----Tweet is neutral, according to the score 1.0
*** {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} ***

RT k2spritebeans nevada be like
----Tweet is neutral, according to the score 0.615
*** {'neg': 0.0, 'neu': 0.615, 'pos': 0.385, 'compound': 0.3612} ***

bio students who arent good at remembering be like
 relearns mitosis and meiosis   relearns mitosis and meiosis… httpstcoNxTlg4swAf
----Tweet is neutral, according to the score 0.765
*** {'neg': 0.115, 'neu': 0.765, 'pos': 0.12, 'compound': 0.0243} ***

RT TXTChartData TXTmembers “Blue Hour” — 0300 KST

24 Old Melon  
64 Genie  
64 VIBE 
95 Bugs  
96 FLO 1
  c…
----Tweet is neutral, according to the score 1.0
*** {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} ***

my bitch she came  with 🦋🦋 a bitch  
----Tweet is negative, according to the score 0.521
*** {'neg': 0.521, 'neu': 0.479, 'pos': 0.0, 'compound': -0.8225} ***

Time 
----Tweet is neutral, according to the score 1.0
*** {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'c

In [27]:
dfTwitter.head()

,created_at,langu,text,source,Result,Analysis Result
0,Wed Nov 04 23:58:20 +0000 2020,NaN,chuuxp pulls out push pop weopon,"<a href=""http://twitter.com/download/android"" ...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral
1,Wed Nov 04 23:58:24 +0000 2020,NaN,RT donelson52 weeps \n\nIndiana college stud...,"<a href=""http://twitter.com/download/iphone"" r...","{'neg': 0.138, 'neu': 0.862, 'pos': 0.0, 'comp...",neutral
2,Wed Nov 04 23:58:27 +0000 2020,NaN,RT xsdvoid ┊┊┊┊⋆ ✧ · ✧ ▪\n┊┊┊☆ ⋆\n┊┊★...,"<a href=""http://twitter.com/download/android"" ...","{'neg': 0.0, 'neu': 0.889, 'pos': 0.111, 'comp...",neutral
3,Wed Nov 04 23:58:27 +0000 2020,NaN,RT tinazhxng if you are from pennsylvania wisc...,"<a href=""http://twitter.com/download/iphone"" r...","{'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'comp...",neutral
4,Wed Nov 04 23:58:28 +0000 2020,NaN,RT k2spritebeans nevada be like,"<a href=""http://twitter.com/download/iphone"" r...","{'neg': 0.0, 'neu': 0.615, 'pos': 0.385, 'comp...",neutral


In [28]:
import json
from pymongo import MongoClient

In [29]:
# how to connect to the mongodb instance on the computer with port and addr
conn = MongoClient('localhost', 27017)

In [30]:
# looking for databases on my mongodb
cursor = conn.list_databases()
for db in cursor:
    print(db)

{'name': 'CarsDB', 'sizeOnDisk': 81920.0, 'empty': False}
{'name': 'Classproject', 'sizeOnDisk': 102400.0, 'empty': False}
{'name': 'Testing_DB', 'sizeOnDisk': 8192.0, 'empty': False}
{'name': 'admin', 'sizeOnDisk': 40960.0, 'empty': False}
{'name': 'config', 'sizeOnDisk': 36864.0, 'empty': False}
{'name': 'local', 'sizeOnDisk': 73728.0, 'empty': False}


In [31]:
# connecting to the database 
db = conn.Classproject

In [32]:
# looking for the collecction on this database
print(db.list_collection_names(include_system_collections=False) )

['Tweets']


In [33]:
# showing all the columns names
dfTwitter.columns

Index(['created_at', 'langu', 'text', 'source', 'Result', 'Analysis Result'], dtype='object')

In [34]:
# Inserting data into the mongodb database
data_dict= dfTwitter.to_dict("records")
db.Tweets.insert_many(data_dict)

In [35]:
# presenting the data from mongodb using the result from neutral evaluation. 
for x in db.Tweets.find({},{"_id":0,"text": 1, "Analysis Result":"neutral"}):
    print(x)

{'text': 'RT DavidCornDC If I have this right\n Philly not fully reporting until after 930 am And 22m mailin ballots to count in PA\n Mich…', 'Analysis Result': 'neutral'}
{'text': 'singtography  i stan alex', 'Analysis Result': 'neutral'}
{'text': 'RT anj3llyfish this is misleading of the ballots that were rejected 23 of those were rejected due to missing signatures nonetheless…', 'Analysis Result': 'neutral'}
{'text': 'RT NinjaKane Cough  America is the joke of the world cough', 'Analysis Result': 'neutral'}
{'text': 'which automatically makes me the best\n\n a hint of a smile ', 'Analysis Result': 'neutral'}
{'text': 'RT NateSilver538 So far Dems Senate hopes\n\n At least one GA runoff possibly two\n Bullock has a shot\n Gideon an underdog IMO but s…', 'Analysis Result': 'neutral'}
{'text': 'epuppie  Teleports to dm  httpstcoQFZuX2hhWS', 'Analysis Result': 'neutral'}
{'text': 'RT LifeNewsHQ BREAKING \n\nJoe Biden claims hes on track to win but Trump is leading in the key battlegroun

In [36]:
from bson.son import SON

In [37]:
pipeline = [
    {"$unwind": "$Analysis Result"},
    {"$group": {"_id": "$Analysis Result", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

In [38]:
import pprint
pprint.pprint(list(db.Tweets.aggregate(pipeline)))

[{'_id': 'neutral', 'count': 872},
 {'_id': 'Compound', 'count': 91},
 {'_id': 'negative', 'count': 22},
 {'_id': 'positive', 'count': 16},
 {'_id': 0, 'count': 1}]


In [39]:
for x in db.Tweets.find({},{"_id":0,"text": 1, "Analysis Result":"negative"}):
    print(x)

{'text': 'RT DavidCornDC If I have this right\n Philly not fully reporting until after 930 am And 22m mailin ballots to count in PA\n Mich…', 'Analysis Result': 'negative'}
{'text': 'singtography  i stan alex', 'Analysis Result': 'negative'}
{'text': 'RT anj3llyfish this is misleading of the ballots that were rejected 23 of those were rejected due to missing signatures nonetheless…', 'Analysis Result': 'negative'}
{'text': 'RT NinjaKane Cough  America is the joke of the world cough', 'Analysis Result': 'negative'}
{'text': 'which automatically makes me the best\n\n a hint of a smile ', 'Analysis Result': 'negative'}
{'text': 'RT NateSilver538 So far Dems Senate hopes\n\n At least one GA runoff possibly two\n Bullock has a shot\n Gideon an underdog IMO but s…', 'Analysis Result': 'negative'}
{'text': 'epuppie  Teleports to dm  httpstcoQFZuX2hhWS', 'Analysis Result': 'negative'}
{'text': 'RT LifeNewsHQ BREAKING \n\nJoe Biden claims hes on track to win but Trump is leading in the key batt